<h2> Populating Data Frame </h2>


Resources:

https://arxiv.org/abs/1403.2345

https://gab41.lab41.org/2-highly-effective-ways-to-estimate-user-location-in-social-media-65eb1e2d8482#.hjsu1opws

In [1]:
import json
import pandas as pd
import bokeh
from bokeh.plotting import figure, output_notebook, show

#Inline Plotting
output_notebook()

Loading BokehJS ...

In [2]:
def load_twitter_data(tweets_data_path):
    tweets_data = []
    tweets_file = open(tweets_data_path, "r")
    for line in tweets_file:
        try:
            tweet = json.loads(line)
            tweets_data.append(tweet)
        except:
            continue
    return tweets_data

def pop_tweets(path):
    tweets_data = load_twitter_data(path)
    tweets = pd.DataFrame(columns=['screenName','userId','text','latt','long','location'])

    for tweet in tweets_data:
        if ('text' in tweet): 
            if tweet['coordinates'] != None:
                tweets.loc[len(tweets)]=[tweet['user']['screen_name'],tweet['user']['id'],tweet['text'], \
                                         tweet['coordinates']['coordinates'][0],\
                                         tweet['coordinates']['coordinates'][1],tweet['place']['full_name']]    
            else:
                tweets.loc[len(tweets)]=[tweet['user']['screen_name'],tweet['user']['id'],tweet['text'], \
                                         float('nan'),float('nan'),tweet['place']['full_name']]    
        
    return tweets

In [3]:
PoGo_tweets = pop_tweets('PoGo_USA.json')

In [4]:
PoGo_tweets.head(n=5)

,screenName,userId,text,latt,long,location
0,desmond_ayala,2.953472e+09,Which pokemon go team did y'all chose? #valor,NaN,NaN,"Caldwell, ID"
1,aphrospice,1.629086e+07,#Magikarp practicing his struggle skills in th...,-73.9187,40.6943,"Brooklyn, NY"
2,ABellgowan,1.681036e+09,Pokemon Go is taking over my life #TeamInstinct,NaN,NaN,"Bixby, OK"
3,JangoSnow,1.057434e+07,Go Team Instinct! I like underdogs. :) https:...,NaN,NaN,"Los Angeles, CA"
4,MetalMushin,6.033327e+08,Gonna go fuck up some #TeamMystic and #TeamIns...,NaN,NaN,"Orange, CT"


<h1> If we need more data: Estimating state from social network for all tweets </h1>
ways to improve this: <br />
Machine learning -  <br />
Include information about the location of state - <br />

In [ ]:
import time
import tweepy
from tweepy import OAuthHandler

consumer_key = 'XxMbTu6bFIU7v4agH3OqRDDYi'
consumer_secret = 'wpF7eKmVDlH3zJzvORWD0Pr6j8dqAhczrIFq4c3ZAH0g3kDw5Q'
access_token = '722589135987478528-SxCJ6LxQaM6A1pa3ZTUj5CYWVZkW48m'
access_secret = 'ChZ6oU0B6ALmrI1bE9GaW9nXD7c8w3nSdtGArwnHAe6Ho'
 
#auth = OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_secret)
#api = tweepy.API(auth)

# Replace the API_KEY and API_SECRET with your application's key and secret.
auth = tweepy.AppAuthHandler('XxMbTu6bFIU7v4agH3OqRDDYi', 'wpF7eKmVDlH3zJzvORWD0Pr6j8dqAhczrIFq4c3ZAH0g3kDw5Q')
 
api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)
 
if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

Find the most likely user state based on most common friend's state (normalized to the population of each state)

In [ ]:
#Dictionary of state populations (in millions) - from google and US census
state_pops = {
        'Alaska' : 0.736,
        'Alabama': 4.849,
        'Arkansas' : 2.966,
        'Arizona': 6.731,
        'California' : 38.8,
        'Colorado' : 5.356,
        'Connecticut' : 3.597,
        'District of Columbia' : 0.659,
        'Delaware' : 0.936,
        'Florida': 19.89,
        'Georgia': 10.1,
        'Hawaii': 1.42,
        'Iowa': 3.107,
        'Idaho' : 1.634,
        'Illinois' : 12.88,
        'Indiana' : 6.597,
        'Kansas' : 2.904,
        'Kentucky' : 4.413,
        'Louisiana' : 4.65,
        'Massachusetts' : 6.745,
        'Maryland' : 5.976,
        'Maine' : 1.33,
        'Michigan' : 9.91,
        'Minnesota' : 5.457,
        'Missouri' : 6.064,
        'Mississippi' : 2.994,
        'Montana' : 1.024,
        'North Carolina' : 9.944,
        'North Dakota' : 0.739,
        'Nebraska' : 1.882,
        'New Hampshire' : 1.327,
        'New Jersey' : 8.938,
        'New Mexico' : 2.086,
        'Nevada' : 2.839,
        'New York' : 19.75,
        'Ohio' : 11.59,
        'Oklahoma' : 3.878,
        'Oregon' : 3.97,
        'Pennsylvania' : 12.79,
        'Puerto Rico' : 3.548,
        'Rhode Island' : 1.055,
        'South Carolina' : 4.832,
        'South Dakota' : 0.853,
        'Tennessee' : 6.549,
        'Texas' : 26.96,
        'Utah' : 2.943,
        'Virginia' : 8.326,
        'Vermont' : 0.627,
        'Washington' : 7.062,
        'Wisconsin' : 5.758,
        'West Virginia' : 1.85,
        'Wyoming' : 0.584
}

In [ ]:
from collections import Counter

def estimateState(userId):
#userId should be in the form int(PoGo_tweets.ix[3,'userId'])
    #Find the user's friends' locations
    friend_locs = []

    #follower_query_remaining = api.rate_limit_status()['resources']['followers']['/followers/list']['remaining']
    #friend_query_remaining = api.rate_limit_status()['resources']['friends']['/friends/list']['remaining']
    
    #switch off between query methods to double our rate limit (now we get 2 queries per min)
    '''
    if follower_query_remaining > friend_query_remaining:
        for user in tweepy.Cursor(api.following, screen_name=PoGo_tweets.ix[idx,'screenName']).items(50):
            if user.location:
                friend_locs.append(user.location)  
    else:
    '''
    for friend in tweepy.Cursor(api.friends, id=userId, count=40).items():
        if friend.location:
            friend_locs.append(friend.location)    
        
    #Parse the list of friend locations
    friend_state_list=[]
    for info in friend_locs:
        new_state = [state for state in state_names if state in info] + \
                    [states[state] for state in states if state in info]

        #Handle cases like "District of Columbia, Washington" and "New York, NY"
        if len(new_state) > 1:
            new_state = new_state[1].split('junk_sadas')

        #Handle cases where no state is mentioned    
        if not new_state:
            new_state = ['None']

        friend_state_list+=new_state

    friend_state_list=[i for i in friend_state_list if i != 'None']
    state_counts=Counter(friend_state_list)
    norm_state_counts={}
    com_state = state_counts.most_common(1)
    
    for state in state_counts:
        norm_state_counts[state] = state_counts[state] / state_pops[state]

    if state_counts:
        return (max(norm_state_counts, key=norm_state_counts.get), str(com_state).split("'")[1])
    else:
        return (None,None)

In [ ]:
api.rate_limit_status()['resources']['friends']['/friends/list']['remaining']

Add estimated state column to dataframe and determine accuracy of estimate.  
NOTE: we can use the accuracy of the state estimate to determine true fraction of each team in a state
using bayesian statistics (see phone pics)


In [ ]:
for idx in range(len(PoGo_tweets)):
    PoGo_tweets.ix[idx,'est_state_norm'], PoGo_tweets.ix[idx,'est_state']=estimateState(int(PoGo_tweets.ix[idx,'userId']))

In [ ]:
PoGo_tweets.head(n=10)

<h1> Trying api.search to get friends instead due to rate limit issues</h1>
Usually not enough @user tweets in the past week to do this

In [ ]:
idx=1

In [ ]:
#Checking if cursor method gives same result
max_tweets = 200
searchQuery = '@%s' % PoGo_tweets.ix[idx,'screenName']
friend_loc = []
for tweet in tweepy.Cursor(api.search,q=searchQuery).items(max_tweets):
    if tweet.place:
        friend_loc.append(tweet.place.full_name)

In [ ]:
len(friend_loc)